# Contextualização do Problema e Solução Proposta
Costumo ler muitos posts do *medium.com* relacionado a ciência de dados, e a escolha de um bom post para ler acaba demandando muito tempo. Com isso, um **sistema de recomendação de posts do medium** com temas relacionados ao meu interesse, acabaria com a enorme quantidade de tempo perdido para escolher um bom post.

Há diversas estratégias para construção de sistemas de recomendação. Existem as baseadas em *filtragem por conteúdo*, *filtragem colaborativa*, porém a que utilizaremos aqui é simplesmente *prever e depois ordenar*.

De acordo com as **informações públicas** do medium, pensamos que o *título* é a principal feature para o modelo, e que algumas informações extras do posts também podem dar uma ajudinha para o modelo, como *quantidade de curtidas*, *comentários*, *tempo requerido para ler o post*, enfim, informações de engajamento em geral.

# Processo
É importante falar qual processo/método guiará esse trabalho. Após a concepção da ideia, debate de principais features a serem capturadas, haverá a exploração e limpeza dos dados, depois a escolha de uma boa métrica, criação de uma baseline, treinamento, otimização dos hiperparâmetros, validação e um pós-processamento nas previsões.

# Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np
import time
import joblib as jb

#VISUALIZAÇÃO
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#SCIPY
from scipy.sparse import hstack, vstack, csr_matrix

#SCIKIT-LEARN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import average_precision_score, roc_curve, roc_auc_score, classification_report, f1_score, precision_recall_curve, confusion_matrix
from sklearn.model_selection import train_test_split, KFold, learning_curve, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

#SCIKIT-OPTIMIZE
from skopt import dummy_minimize, gp_minimize, BayesSearchCV
from skopt.plots import plot_convergence

from lightgbm import LGBMClassifier

In [2]:
df = pd.read_csv("dataset_clean.csv")

In [3]:
df.head(2)

,Unnamed: 0,comments,key_word,palms,reading_time,title,target,log_comments,log_palms,log_reading_time
0,0,64,Data Science,21000.0,9.0,learn data science broke,1,1.812913,4.322240,1.000000
1,1,77,Data Science,17800.0,18.0,build data science portfolio,1,1.892095,4.250444,1.278754


In [4]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [5]:
df.shape

(1465, 9)

# Escolha da Métrica
A escolha da métrica é **fundamental** para o sucesso do seu projeto! Com a escolha errada da métrica, é bem provável que você otimize o seu modelo para o problema errado, implicando em gasto de tempo, recursos financeiros e até mesmo o fracasso do projeto.

Para esse problema, levando em consideração que é um problema binário, levemente desbalanceado, e como os FP (Falsos Positivos) e VP (Verdadeiros Positivos) são importantes, optei pela métrica **Average Precision** (Precisão Média) não interpolada.

# Train/Test
Geralmente, a porcentagem indicada na literatura para separar os dados são 70% (train) e 30% (test), outras indicam 80/20... Mas isso **depende** muito da quantidade dos dados que se para trabalhar. A escolha é sempre um **trade-off**, pois mais dados para validação gera mais *confiabilidade* na avaliação do modelo. Já mais dados para treinamento, implica em mais exemplos para o modelo *aprender* melhor (generalizar). Optei por 70/30, pela quantidade de dados em questão, e o processo de validação cruzada adiante.

In [6]:
features = ['comments', 'palms', 'reading_time', 'title']
target = df['target']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df[features], target, test_size=0.3, random_state=0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1025, 4) (440, 4) (1025,) (440,)


Como a *feature title* é um texto, optei por uma estratégia comum e simples para uma primeira solução, que é o **TF-IDF** (Term Frequency-Inverse Document Frequency). É uma medida estatística que avalia a relevância de uma palavra de um documento em relação a um conjunto de documentos.

In [8]:
title_train = X_train['title']
title_test = X_test['title']

In [9]:
title_vec = TfidfVectorizer(min_df=1, ngram_range=(1,1))
title_bow_train = title_vec.fit_transform(title_train)
title_bow_test = title_vec.transform(title_test)

In [10]:
print(title_bow_train.shape, title_bow_test.shape)

(1025, 1791) (440, 1791)


In [11]:
X_train_wtitle = hstack([X_train[['comments', 'palms', 'reading_time']], title_bow_train])
X_test_wtitle = hstack([X_test[['comments', 'palms', 'reading_time']], title_bow_test])
print(X_train_wtitle.shape, X_test_wtitle.shape)

(1025, 1794) (440, 1794)


In [12]:
X_train_wtitle2 = csr_matrix(X_train_wtitle.copy())
X_test_wtitle2 = csr_matrix(X_test_wtitle.copy())

scaler = MaxAbsScaler()

X_train_wtitle2[:, :3] = scaler.fit_transform(X_train_wtitle2[:, :3].todense())
X_test_wtitle2[:, :3] = scaler.transform(X_test_wtitle2[:, :3].todense())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  import sys


# Baseline
Uma linha de base é uma solução simples e fácil de ser implementada. É importante para fornecer um ponto de base para comparação com os modelos criados com aprendizado de máquina. Algumas estratégias básicas de baseline para problemas de classificação binária são:
* **Palpite Aleatório Uniforme** (Prever 0 ou 1 com probabilidades iguais).
* **Palpite Aleatório Anterior** (Prever 0 ou 1 proporcional à probabilidade anterior no conjunto de dados)
* **Classe Majoritária** (Prever a classe de maior frequência)
* **Classe Minoritária** (Prever a classe de menor frequência)
* **Classe Anterior** (Prever a probabilidade anterior a cada classe)

Optei pela classe majoritária, por influência da escolha da métrica escolhida para esse problema.

In [13]:
print(average_precision_score(y_test, np.ones(440))) 

0.6431818181818182


# Treinamento e Validação
Nessa etapa, foi realizado uma **busca otimizada bayesiana**, que se mostra geralmente mais eficiente do que outras estratégias de busca para *tuning de hiperparâmetros*. Optei pelos algoritmos *Random Forest*, *Logistic Regression* e *LGBM* nessa primeira solução. Foi realizado uma média aritmética nas previsões, com o intuito de uma estabilidade maior, do que apenas um modelo para decidir sozinho.

In [14]:
ITERATIONS = 30

cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=42
    )

param_grid_RF = {
    'n_estimators': (50, 1000),
    'min_samples_leaf': (1, 128),
    'min_samples_split': (2, 128),
    'max_depth': (1, 128),
    'max_features': ['sqrt', 'log2']
}

param_grid_LR = {
    'tol': (1e-4, 1e-3, 'log-uniform'),
    'C': (1e-4, 8),
    'fit_intercept': [True, False],
    'max_iter': (50, 1100)
}

param_grid_LGBM = {
    'learning_rate': (1e-4, 1e-0, 'log-uniform'),
    'n_estimators': (50, 1000),
    'colsample_bytree': (0.1, 1.0),
    'min_child_samples': (1, 100),
    'num_leaves': (2, 128),
    'subsample': (0.05, 1.0)
}

In [15]:
def run_fit(param_grid, model, X_train, y_train, num_iter=30, cv=3, random_state=5):
    
    opt = BayesSearchCV(
            model,
            param_grid,
            scoring='average_precision',
            n_iter=num_iter,
            random_state=random_state,
            verbose=0,
            cv=cv
            )
    print("Total de buscas:", opt.total_iterations)
    print("Treinando...")
    opt.fit(X_train, y_train)
    print("Pontuações da Precisão Média (CV nos dados de Treino):", np.mean(opt.cv_results_['mean_test_score']))
    print("Média das Pontuações da Precisão Média (CV nos dados de Treino):", np.mean(opt.cv_results_['mean_test_score']))
    print("Desvio Padrão Precisão Média (CV nos dados de Treino):", np.mean(opt.cv_results_['std_test_score']))
    return opt.best_estimator_, opt.best_params_

In [16]:
inicio = time.time()
rf, rf_params = run_fit(param_grid_RF, RandomForestClassifier(class_weight='balanced', random_state=0, verbose=0),
             X_train_wtitle2, y_train, num_iter=ITERATIONS, cv=cv)
fim = time.time()
preds_rf = rf.predict_proba(X_test_wtitle2)[:, 1]
print("Precisão Média (nos dados de Test):", average_precision_score(y_test, preds_rf))
print("Tempo de Execução:", (fim - inicio)/60) 

Total de buscas: 150
Treinando...
Pontuações da Precisão Média (CV nos dados de Treino): 0.7275142133419121
Média das Pontuações da Precisão Média (CV nos dados de Treino): 0.7275142133419121
Desvio Padrão Precisão Média (CV nos dados de Treino): 0.017924018207996585
Precisão Média (nos dados de Test): 0.8191559292072723
Tempo de Execução: 5.86496924161911


In [17]:
inicio = time.time()
lr, lr_params = run_fit(param_grid_LR, LogisticRegression(class_weight='balanced', random_state=0, verbose=0),
             X_train_wtitle2, y_train, num_iter=ITERATIONS, cv=cv)
fim = time.time()

preds_lr = lr.predict_proba(X_test_wtitle2)[:, 1]
print("Precisão Média (nos dados de Test):", average_precision_score(y_test, preds_lr))
print("Tempo de Execução:", (fim - inicio)/60) 

Total de buscas: 120
Treinando...
Pontuações da Precisão Média (CV nos dados de Treino): 0.78712363541303
Média das Pontuações da Precisão Média (CV nos dados de Treino): 0.78712363541303
Desvio Padrão Precisão Média (CV nos dados de Treino): 0.018635157713820592
Precisão Média (nos dados de Test): 0.842546223161273
Tempo de Execução: 0.6820167024930318


In [18]:
inicio = time.time()
lgbm, lgbm_params = run_fit(param_grid_LGBM, LGBMClassifier(class_weight='balanced', subsample_freq=1,
                                                            random_state=0, verbose=0), X_train_wtitle2, y_train,num_iter=40, cv=cv)
fim = time.time()

preds_lgbm = lgbm.predict_proba(X_test_wtitle2)[:, 1]
print("Precisão Média (nos dados de Test):", average_precision_score(y_test, preds_lgbm))
print("Tempo de Execução:", (fim - inicio)/60)

Total de buscas: 240
Treinando...


C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Pontuações da Precisão Média (CV nos dados de Treino): 0.683822188590133
Média das Pontuações da Precisão Média (CV nos dados de Treino): 0.683822188590133
Desvio Padrão Precisão Média (CV nos dados de Treino): 0.01815178221965385
Precisão Média (nos dados de Test): 0.8108739205879548
Tempo de Execução: 10.184617924690247


In [19]:
pd.DataFrame({"LR": preds_lr, "RF": preds_rf, "LGBM": preds_lgbm}).corr()

,LR,RF,LGBM
LR,1.000000,0.790992,0.676033
RF,0.790992,1.000000,0.848669
LGBM,0.676033,0.848669,1.000000


In [20]:
p = (preds_lr + preds_rf + preds_lgbm)/3
average_precision_score(y_test, p)

0.8443425544695263

# Modelos Finais
Depois de otimizar os hiperparâmetros dos modelos, optei por realizar o treinamento com todos os dados, afim de obter uma melhor generalização em produção.

In [21]:
title_vec_final = TfidfVectorizer(min_df=1, ngram_range=(1,1))
title_bow_final = title_vec_final.fit_transform(df['title'])

scaler_final = MaxAbsScaler()
X_scaled = scaler_final.fit_transform(df[['comments', 'palms', 'reading_time']])

X_final = hstack([X_scaled, title_bow_final])
print(X_final.shape, target.shape)

(1465, 2232) (1465,)


In [22]:
rf_final = RandomForestClassifier(**rf_params)
rf_final.fit(X_final, target)

lr_final = LogisticRegression(**lr_params)
lr_final.fit(X_final, target)

lgbm_final = LGBMClassifier(**lgbm_params)
lgbm_final.fit(X_final, target)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.1,
               importance_type='split', learning_rate=0.00031605447203434024,
               max_depth=-1, min_child_samples=1, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=1000, n_jobs=-1, num_leaves=128,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

**Salvando os vetorizadores, transformadores e modelos.**

In [23]:
jb.dump(title_vec_final, "title_vectorizer_20200803.pkl.z")
jb.dump(scaler_final, "scaler_20200803.pkl.z")
jb.dump(rf_final, "random_forest_20200803.pkl.z")
jb.dump(lr_final, "logistic_reg_20200803.pkl.z")
jb.dump(lgbm_final, "lgbm_20200803.pkl.z")

['lgbm_20200803.pkl.z']